# Setup Folders

In [1]:
import os

data_folder = "./data"
os.makedirs(data_folder, exist_ok=True)

# Settings

In [2]:
OBSERVATION_CROP_DIM = 83
OBSERVATION_DIM = 64

LOG_LEVEL = "INFO"

# Install and Load Packages

In [3]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
import itertools
import json
import multiprocessing
import os
from typing import Dict, Any

from tqdm.notebook import tqdm

from src.car_racing_worker import run_single_exploration

In [5]:
import logging
import coloredlogs

logger = logging.getLogger(__name__)
coloredlogs.install(level=LOG_LEVEL, logger=logger, fmt="%(asctime)s [%(levelname)s] %(message)s", isatty=True)
logger.info("Logger initialized.")

2025-12-02 20:09:35 [INFO] Logger initialized.


In [6]:
os.environ["PYTHONWARNINGS"] = "ignore::UserWarning" # Pygame has a deprecation warning at the moment

# Setup

In [7]:
def ensure_count_file_exists(data_folder):
  run_count_file_path = os.path.join(data_folder, "run_count.txt")
  if not os.path.exists(run_count_file_path):
    with open(run_count_file_path, 'w') as count_file:
      count = str(0).zfill(7)
      count_file.write(str(count))


In [8]:
ensure_count_file_exists(data_folder)

# Functions

In [9]:
def get_current_count(data_folder):
  run_count_file_path = os.path.join(data_folder, "run_count.txt")
  try:
    with open(run_count_file_path, "r") as count_file:
      count_str = count_file.read().strip()
      if count_str:
        return int(count_str)
      return 0
  except (FileNotFoundError, ValueError):
    return 0

In [10]:
def load_metadata(data_folder: str) -> Dict[str, Dict[str, Any]]:
    metadata = {}
    metadata_path = os.path.join(data_folder, "metadata.json")
    if os.path.exists(metadata_path):
        with open(metadata_path, "r") as file:
            metadata = json.load(file)
    return metadata

def save_metadata(metadata: Dict[str, Dict[str, Any]], data_folder: str) -> None:
    metadata_path = os.path.join(data_folder, "metadata.json")
    with open(metadata_path, "w") as file:
        json.dump(metadata, file)

In [11]:
def run_multiple_explorations(data_folder, num_processes, target_count, logger):
    logger.info(f"Starting {num_processes} parallel processes")
    start_count = get_current_count(data_folder)
    if start_count > target_count:
        logger.warning(f"Current next exploration ({start_count}) already meets or exceeds target ({target_count})")
        return
    
    metadata = load_metadata(data_folder)
    ctx = multiprocessing.get_context('spawn')
    with ctx.Pool(processes=num_processes) as pool:
        kwargs = {
            "data_folder": data_folder,
            "y_crop_dim": OBSERVATION_CROP_DIM,
            "observation_dim": OBSERVATION_DIM,
            "logger": logger
        }
        tasks_iterable = itertools.repeat(kwargs)
        with tqdm(total=target_count, desc="Explorations Completed") as pbar:
            pbar.n = start_count
            pbar.refresh()
            result_pool = pool.imap_unordered(run_single_exploration, tasks_iterable)
            for result_count, result_file_name, result_length in result_pool:
                logger.debug(result_count)
                logger.debug(result_file_name)
                logger.debug(result_length)
                metadata[result_file_name] = {"length": result_length}
                if result_count >= target_count:
                    logger.debug(f"\nReached target count {result_count}")
                    logger.debug("Terminating all worker processes...")
                    pool.terminate()
                    pool.join()
                    logger.info(f"Finished all {target_count} explorations")
                    logger.debug(metadata)
                    save_metadata(metadata, data_folder)
                    return
                pbar.n = result_count + 1
                pbar.refresh()

# Run

In [12]:
run_multiple_explorations(data_folder, num_processes=8, target_count=1500, logger=logger)

2025-12-02 20:09:35 [INFO] Starting 8 parallel processes


Explorations Completed:   0%|          | 0/1500 [00:00<?, ?it/s]

2025-12-02 20:33:44 [INFO] Finished all 1500 explorations
